In [19]:
# import modules
import os
import pandas as pd
import numpy as np
import random
import pickle

# import scikit-learn modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, roc_auc_score, average_precision_score, precision_recall_curve, auc, roc_curve

# import visualization modules
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
cwd = os.getcwd()
BASE_DIR = os.path.abspath(os.path.join(cwd, "..", ".."))

# build paths inside the repo
get_data_path = lambda folders, fname: os.path.normpath(
    os.path.join(BASE_DIR, *folders, fname)
)

file_path_training_data = get_data_path(['data', 'output', 'models'], 'training_data.csv')
klingbeil_file_path = get_data_path(['data', 'output', 'models'], 'testing_data.csv')
parrish_file_path = get_data_path(['data', 'output', 'models'], 'testing_data_parrish.csv')

# Output
file_RF_model_contextualised = get_data_path(['data', 'output', 'models'], 'contextualised_model.pickle')
file_RF_model_full = get_data_path(['data', 'output', 'models'], 'full_model.pickle')

figure_path_roc_auc = get_data_path(['figures'], '4a_roc_auc.png')
figure_path_pr_auc = get_data_path(['figures'], '4b_pr_auc.png')

figure_path_parrish_roc_auc = get_data_path(['figures'], '4a_roc_auc_parrish.png')
figure_path_parrish_pr_auc = get_data_path(['figures'], '4b_pr_auc_parrish.png')


In [21]:
training_df = pd.read_csv(file_path_training_data)
training_df[:3]

,genepair,A1,A2,A1_entrez,A2_entrez,DepMap_ID,cell_line,Gemini_FDR,raw_LFC,SL,...,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,GEMINI,LFC,SL_new
0,A3GALT2_ABO,A3GALT2,ABO,127550.0,28.0,ACH-000022,PATU8988S_PANCREAS,0.998944,0.088856,False,...,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.118768,0.088856,False
1,A3GALT2_ABO,A3GALT2,ABO,127550.0,28.0,ACH-000307,PK1_PANCREAS,0.986587,0.201704,False,...,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.132501,0.201704,False
2,A3GALT2_ABO,A3GALT2,ABO,127550.0,28.0,ACH-000632,HS944T_SKIN,1.000000,0.069772,False,...,0.0,False,3.0,0.0,0.114847,0.258739,11.702,0.024593,0.069772,False


In [22]:
test_df = pd.read_csv(klingbeil_file_path)
test_df[:3]

,GENE_COMBINATION,domain_combination,genepair,A1,A2,A1_entrez,A2_entrez,cell_line,DepMap_ID,GEMINI,...,either_in_complex,mean_complex_essentiality,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,SL_new
0,AAK1:Kinase_domain;BMP2K:Kinase_domain,Kinase_domain_Kinase_domain,AAK1_BMP2K,AAK1,BMP2K,22848.0,55589.0,HEL,ACH-000004,0.218665,...,False,0.0,0.0,False,77.0,21.867726,0.261701,6.713555,6.761786,False
1,AAK1:Kinase_domain;BMP2K:Kinase_domain,Kinase_domain_Kinase_domain,AAK1_BMP2K,AAK1,BMP2K,22848.0,55589.0,T3M4,ACH-000085,0.205641,...,False,0.0,0.0,False,77.0,21.867726,0.261701,6.713555,6.761786,False
2,AAK1:Kinase_domain;BMP2K:Kinase_domain,Kinase_domain_Kinase_domain,AAK1_BMP2K,AAK1,BMP2K,22848.0,55589.0,HPAFII,ACH-000094,0.044486,...,False,0.0,0.0,False,77.0,21.867726,0.261701,6.713555,6.761786,False


In [23]:
test_df_parrish = pd.read_csv(parrish_file_path)
test_df_parrish[:3]

,genepair,A1,A2,A1_entrez,A2_entrez,PC9_GI_score,PC9_GI_fdr,HeLa_GI_score,HeLa_GI_fdr,DepMap_ID,...,colocalisation,interact,n_total_ppi,fet_ppi_overlap,gtex_spearman_corr,gtex_min_mean_expr,gtex_max_mean_expr,GEMINI,LFC,SL_new
0,A2M_PZP,A2M,PZP,2.0,5858.0,0.264313,0.138809,-0.154320,0.424612,ACH-000779,...,0.0,True,122.0,2.97235,0.601651,0.804023,473.357464,-0.249934,0.333503,False
1,A2M_PZP,A2M,PZP,2.0,5858.0,0.264313,0.138809,-0.154320,0.424612,ACH-001086,...,0.0,True,122.0,2.97235,0.601651,0.804023,473.357464,-0.038689,-0.115678,False
2,AADACL3_AADACL4,AADACL3,AADACL4,126767.0,343066.0,-0.000281,0.992873,0.120862,0.433194,ACH-000779,...,0.0,False,0.0,0.00000,0.141775,0.213038,0.350221,0.022368,-0.106713,False


In [24]:
feature_columns_1 = ['rMaxExp_A1A2', 'rMinExp_A1A2',
                     'max_ranked_A1A2', 'min_ranked_A1A2',
                     'max_cn', 'min_cn', 'Protein_Altering', 'Damaging', 
                     'min_sequence_identity',
                     'prediction_score', 
                     'weighted_PPI_essentiality', 'weighted_PPI_expression',
                     'smallest_BP_GO_essentiality', 'smallest_CC_GO_essentiality',
                     'smallest_BP_GO_expression', 'go_CC_expression'
                     ]

target_column = 'SL_new'

print('num of features:', len(feature_columns_1))

num of features: 16


In [25]:
feature_columns_2 = feature_columns_1 + ['closest', 'WGD', 'family_size',
                                         'cds_length_ratio', 'shared_domains', 'has_pombe_ortholog',
                                         'has_essential_pombe_ortholog', 'has_cerevisiae_ortholog',
                                         'has_essential_cerevisiae_ortholog', 'conservation_score', 'mean_age',
                                         'either_in_complex', 'mean_complex_essentiality', 'colocalisation',
                                         'interact', 'n_total_ppi', 'fet_ppi_overlap',
                                         'gtex_spearman_corr', 'gtex_min_mean_expr', 'gtex_max_mean_expr']
feature_columns_2.remove('prediction_score')
print('num of features:', len(feature_columns_2))

num of features: 35


### Prepare Seen and Unseen subsets

In [26]:
# splitting data for Klingbeil Screen

def split_seen_unseen(test_df, training_df):
    """
    Split test_df into seen and unseen pairs based on genepair only.
    Returns: seen_df, unseen_df
    """
    common_cell_line = set(training_df['DepMap_ID'].unique()).intersection(set(test_df['DepMap_ID'].unique()))
    novel_test_df = test_df[~test_df['DepMap_ID'].isin(common_cell_line)]

    train_genepairs = set(training_df['genepair'])
    seen_mask = novel_test_df['genepair'].isin(train_genepairs)
    unseen_mask = ~seen_mask

    seen_df = novel_test_df[seen_mask].copy()
    unseen_df = novel_test_df[unseen_mask].copy()

    print(f"Seen pairs (by genepair): {seen_df.shape[0]}")
    print(f"Unseen pairs (by genepair): {unseen_df.shape[0]}")
    return seen_df, unseen_df

In [27]:
klingbeil_seen, klingbeil_unseen = split_seen_unseen(test_df, training_df)

Seen pairs (by genepair): 21826
Unseen pairs (by genepair): 14672


In [28]:
# splitting data for Parrish Screen

def split_seen_unseen_by_genepair(test_df, training_df):
    """
    Split test_df into seen and unseen pairs based on genepair only.
    Returns: seen_df, unseen_df
    """
    train_genepairs = set(training_df['genepair'])
    seen_mask = test_df['genepair'].isin(train_genepairs)
    unseen_mask = ~seen_mask

    seen_df = test_df[seen_mask].copy()
    unseen_df = test_df[unseen_mask].copy()

    print(f"Seen pairs (by genepair): {seen_df.shape[0]}")
    print(f"Unseen pairs (by genepair): {unseen_df.shape[0]}")
    return seen_df, unseen_df

In [29]:
parrish_seen, parrish_unseen = split_seen_unseen_by_genepair(test_df_parrish, training_df)

Seen pairs (by genepair): 549
Unseen pairs (by genepair): 1216


### Run RF Classifier


In [30]:
def calc_auc(df, RF, feature_columns, target_column):
    ypred = RF.predict_proba(df[feature_columns])[:,1]
    auc_pr_df = pd.DataFrame({
        'SL':df[target_column],
        'ypred':ypred,
        'prediction_score':df['prediction_score'],
        'rMinExp_A1A2': df['rMinExp_A1A2'],
        'min_ranked_A1A2': df['min_ranked_A1A2'],
        'transformed_gene_essentiality': -df['min_ranked_A1A2'],
        'min_sequence_identity': df['min_sequence_identity']})
    return auc_pr_df

In [31]:
# Define feature sets
data_1 = training_df[feature_columns_1]
data_2 = training_df[feature_columns_2]
target = training_df[target_column]

In [32]:
RF = RandomForestClassifier(n_estimators=600, random_state=8, max_features=0.2, max_depth=20, min_samples_leaf=4)

In [ ]:
contextualised = RF.fit(data_1, target)

In [ ]:
with open(file_RF_model_contextualised, 'wb') as f:
    pickle.dump(contextualised, f)

In [33]:
# Load RF classifier from file
with open(file_RF_model_contextualised, 'rb') as file:
    contextualised = pickle.load(file)

In [ ]:
full = RF.fit(data_2, target)

In [ ]:
with open(file_RF_model_full, 'wb') as f:
    pickle.dump(full, f)

In [34]:
# Load RF classifier from file
with open(file_RF_model_full, 'rb') as file:
    full = pickle.load(file)

In [35]:
# Collect all validation datasets
validation_datasets = [
    ("Klingbeil Seen", klingbeil_seen),
    ("Klingbeil Unseen", klingbeil_unseen),
    ("Parrish Seen", parrish_seen),
    ("Parrish Unseen", parrish_unseen)
]

# Evaluate contextualised model
print("Evaluating contextualised Integration Model...")
auc_pr_data_all_contextualised = []
for name, dataset in validation_datasets:
    if len(dataset) > 0:
        print(f"  Processing {name}: {len(dataset)} samples")
        result = calc_auc(dataset, contextualised, feature_columns_1, target_column)
        auc_pr_data_all_contextualised.append(result)
    else:
        print(f"Skipping {name}: No samples")
        empty_df = pd.DataFrame(columns=['SL', 'ypred', 'prediction_score',
                                         'rMinExp_A1A2', 'min_ranked_A1A2',
                                         'transformed_gene_essentiality',
                                         'min_sequence_identity'])
        auc_pr_data_all_contextualised.append(empty_df)

# Evaluate full model
print("\nEvaluating Full Integration Model...")
auc_pr_data_all_full = []
for name, dataset in validation_datasets:
    if len(dataset) > 0:
        print(f"  Processing {name}: {len(dataset)} samples")
        result = calc_auc(dataset, full, feature_columns_2, target_column)
        auc_pr_data_all_full.append(result)
    else:
        print(f"Skipping {name}: No samples")
        empty_df = pd.DataFrame(columns=['SL', 'ypred', 'prediction_score',
                                         'rMinExp_A1A2', 'min_ranked_A1A2',
                                         'transformed_gene_essentiality',
                                         'min_sequence_identity'])
        auc_pr_data_all_full.append(empty_df)

# Prepare validation_data_df for visualization
print("\nPREPARING VALIDATION DATA FOR VISUALIZATION")

validation_data_df = []
dataset_names = ["Klingbeil Seen", "Klingbeil Unseen", "Parrish Seen", "Parrish Unseen"]

for i, name in enumerate(dataset_names):
    if len(auc_pr_data_all_contextualised[i]) > 0 and len(auc_pr_data_all_full[i]) > 0:
        contextualised_df = auc_pr_data_all_contextualised[i].loc[:, ['SL', 'ypred', 'prediction_score', 'min_sequence_identity']]
        full_df = auc_pr_data_all_full[i].loc[:, ['SL', 'ypred']]
        concat_df = pd.concat([
            contextualised_df.rename(columns={'SL': 'SL_contextualised', 'ypred': 'ypred_contextualised'}),
            full_df.rename(columns={'SL': 'SL_full', 'ypred': 'ypred_full'})
        ], axis=1)
        validation_data_df.append(concat_df)
        print(f"{name}: {len(concat_df)} samples prepared")
    else:
        print(f"{name}: Skipped due to insufficient data")
        empty_df = pd.DataFrame(columns=['SL_contextualised', 'ypred_contextualised', 'SL_full', 'ypred_full',
                                         'prediction_score', 'min_sequence_identity'])
        validation_data_df.append(empty_df)

print(f"\nTotal validation sets prepared: {len([df for df in validation_data_df if len(df) > 0])}")

Evaluating contextualised Integration Model...
  Processing Klingbeil Seen: 21826 samples
  Processing Klingbeil Unseen: 14672 samples
  Processing Parrish Seen: 549 samples
  Processing Parrish Unseen: 1216 samples

Evaluating Full Integration Model...
  Processing Klingbeil Seen: 21826 samples
  Processing Klingbeil Unseen: 14672 samples
  Processing Parrish Seen: 549 samples
  Processing Parrish Unseen: 1216 samples

PREPARING VALIDATION DATA FOR VISUALIZATION
Klingbeil Seen: 21826 samples prepared
Klingbeil Unseen: 14672 samples prepared
Parrish Seen: 549 samples prepared
Parrish Unseen: 1216 samples prepared

Total validation sets prepared: 4


In [36]:
# Add comprehensive performance summary:

def generate_validation_summary():
    """Generate a comprehensive summary of validation performance."""
    print("\n📊 VALIDATION PERFORMANCE SUMMARY")
    print("=" * 60)
    
    summary_data = []
    
    for i, (name, df) in enumerate(zip(dataset_names, validation_data_df)):
        if len(df) > 0:
            # Calculate metrics for each approach
            try:
                # contextualised model ROC AUC
                contextualised_roc = roc_auc_score(df['SL_contextualised'], df['ypred_contextualised'])
                contextualised_pr = average_precision_score(df['SL_contextualised'], df['ypred_contextualised'])
                
                # full model ROC AUC  
                full_roc = roc_auc_score(df['SL_full'], df['ypred_full'])
                full_pr = average_precision_score(df['SL_full'], df['ypred_full'])
                
                # Baseline methods
                pred_score_roc = roc_auc_score(df['SL_contextualised'], df['prediction_score'])
                pred_score_pr = average_precision_score(df['SL_contextualised'], df['prediction_score'])
                
                seq_id_roc = roc_auc_score(df['SL_contextualised'], df['min_sequence_identity'])
                seq_id_pr = average_precision_score(df['SL_contextualised'], df['min_sequence_identity'])
                
                summary_data.append({
                    'Dataset': name,
                    'Samples': len(df),
                    'SL_Rate': df['SL_contextualised'].mean(),
                    'contextualised_ROC': contextualised_roc,
                    'contextualised_PR': contextualised_pr,
                    'full_ROC': full_roc,
                    'full_PR': full_pr,
                    'PredScore_ROC': pred_score_roc,
                    'PredScore_PR': pred_score_pr,
                    'SeqID_ROC': seq_id_roc,
                    'SeqID_PR': seq_id_pr
                })
                
            except Exception as e:
                print(f"Error calculating metrics for {name}: {e}")
    
    # Create summary DataFrame
    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        
        print("🎯 ROC AUC Performance:")
        print("-" * 40)
        for _, row in summary_df.iterrows():
            print(f"{row['Dataset']:<15} contextualised: {row['contextualised_ROC']:.3f} | full: {row['full_ROC']:.3f} | PredScore: {row['PredScore_ROC']:.3f} | SeqID: {row['SeqID_ROC']:.3f}")
        
        print("\n📏 PR AUC Performance:")
        print("-" * 40)
        for _, row in summary_df.iterrows():
            print(f"{row['Dataset']:<15} contextualised: {row['contextualised_PR']:.3f} | full: {row['full_PR']:.3f} | PredScore: {row['PredScore_PR']:.3f} | SeqID: {row['SeqID_PR']:.3f}")
        
        return summary_df
    else:
        print("No valid data for summary")
        return None

# Generate the summary
validation_summary_df = generate_validation_summary()


📊 VALIDATION PERFORMANCE SUMMARY
🎯 ROC AUC Performance:
----------------------------------------
Klingbeil Seen  contextualised: 0.869 | full: 0.872 | PredScore: 0.728 | SeqID: 0.571
Klingbeil Unseen contextualised: 0.808 | full: 0.814 | PredScore: 0.725 | SeqID: 0.541
Parrish Seen    contextualised: 0.750 | full: 0.743 | PredScore: 0.672 | SeqID: 0.590
Parrish Unseen  contextualised: 0.750 | full: 0.751 | PredScore: 0.713 | SeqID: 0.533

📏 PR AUC Performance:
----------------------------------------
Klingbeil Seen  contextualised: 0.315 | full: 0.331 | PredScore: 0.150 | SeqID: 0.064
Klingbeil Unseen contextualised: 0.216 | full: 0.241 | PredScore: 0.126 | SeqID: 0.060
Parrish Seen    contextualised: 0.347 | full: 0.377 | PredScore: 0.225 | SeqID: 0.156
Parrish Unseen  contextualised: 0.260 | full: 0.248 | PredScore: 0.187 | SeqID: 0.088


## Draw ROC and PR Curves

In [37]:
def plot_roc_curve(ax, data, true_label, probas, label, color,
                   line_style="solid", draw_style="steps-post", zorder=10, labels_only=False):

    # Drop rows with NaNs in true or predicted columns
    valid_data = data[[true_label, probas]].dropna()

    # Make sure we have enough valid data to compute ROC
    y_true = valid_data[true_label].values
    y_score = valid_data[probas].values

    if len(np.unique(y_true)) < 2:
        print(f"Skipping {label}: not enough class variation after dropping NaNs.")
        return

    # Calculate ROC and AUC
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    legend_label = f"{roc_auc:.2f}"
    ax.step(fpr, tpr, label=legend_label, color=color,
            linestyle=line_style, drawstyle=draw_style, zorder=zorder, linewidth=1.25)

def draw_roc_curves(data, true_labels, probas, labels, title, colors, ax, labels_only=False):
    for true_label, probas, label, color in zip(true_labels, probas, labels, colors):
        plot_roc_curve(ax, data, true_label, probas, label, color, labels_only=labels_only)

    ax.plot([0, 1], [0, 1], linestyle='--', color='gray', label='0.50')
    # ax.set_xlabel('False Positive Rate', fontsize=16)
    # ax.set_ylabel('True Positive Rate', fontsize=16)
    # ax.set_title(f'{title}', fontsize=14)
    # ax.tick_params(axis='x', labelsize=8)
    # ax.tick_params(axis='y', labelsize=8)
    # ax.spines.top.set(visible=False)
    # ax.spines.right.set(visible=False)
    # ax.set_ylim([-0.025, 1.025])
    # ax.set_xlim([-0.025, 1.025])
    # ax.legend(loc='lower right', fontsize=16)

    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title(f'{title}', fontsize=12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.spines.top.set(visible=False)
    ax.spines.right.set(visible=False)
    ax.set_ylim([-0.025, 1.025])
    ax.set_xlim([-0.025, 1.025])
    ax.legend(loc='lower right', fontsize=10)

def draw_validation_curves(validation_data_df, title_list, save_path:None):
    sns.set_context('paper')
    fig, ax = plt.subplots(1, 2, figsize=(5.85, 3.75))

    true_labels = ['SL_contextualised', 'SL_full', 'SL_contextualised', 'SL_contextualised']
    predictions = ['ypred_contextualised', 'ypred_full', 'prediction_score', 'min_sequence_identity']
    custom_labels = ['Context-Specific Classifier (contextualised)', 'Context-Specific Classifier (full)', 'General Classifier', 'Sequence Identity']
    colors = ['#009E73', '#CC79A7', '#777777', '#E69F00']

    for i, df in enumerate(validation_data_df):
        draw_roc_curves(df, true_labels, predictions, custom_labels, title_list[i], colors, ax[i])

    # Extract handles and labels from the first subplot
    handles, labels = ax[0].get_legend_handles_labels()
    
    # Create a shared legend for the figure
    #fig.legend(handles=handles, labels=custom_labels + ['Chance'], ncol=5, loc='upper center', 
    #           bbox_to_anchor=[0.5, 1.2], fontsize=20, fancybox=True, facecolor='#efefef', borderpad=0.5)

    #fig.legend(handles=handles, labels=custom_labels + ['Chance'], ncol=5, loc='upper center', 
    #           bbox_to_anchor=[0.5, 1.1], fontsize=12, fancybox=True, facecolor='#efefef', borderpad=0.5)

    plt.tight_layout(h_pad=0.5)

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=500)
    plt.close()
    #plt.show()

In [ ]:
# Klingbeil only (indices 0 and 1)
draw_validation_curves(validation_data_df[0:2], 
                       ["Klingbeil et al. - Seen", "Klingbeil et al. - Unseen"],
                       figure_path_roc_auc)

In [39]:
# Parrish only (indices 2 and 3)
draw_validation_curves(validation_data_df[2:4], 
                       ["Parrish et al. - Seen", "Parrish et al. - Unseen"],
                       figure_path_parrish_roc_auc)

In [38]:
def plot_pr_curve(ax, data, true_label, probas, label, color,
                  line_style="solid", draw_style="default", zorder=10,
                  labels_only=False, subplot_index=None):
    # Drop rows with NaNs in either column
    valid_data = data[[true_label, probas]].dropna()

    y_true = valid_data[true_label].values
    y_score = valid_data[probas].values

    # Skip if there aren't enough valid samples or variation in labels
    if len(y_true) < 2 or len(np.unique(y_true)) < 2:
        print(f"Skipping {label}: insufficient valid data.")
        return

    precision, recall, _ = precision_recall_curve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    legend_label = f"{ap_score:.2f}"

    ax.plot(recall, precision, label=legend_label, color=color,
            linestyle=line_style, drawstyle=draw_style, zorder=zorder, linewidth=1.25)

    # Optional: annotate with X marker on a fixed point
    # if subplot_index in [2, 3]:
    #     ax.scatter(0.325, 0.4, color='red', marker="X", s=70, zorder=15)


def draw_pr_curves(data, true_labels, probas, labels, title, colors, ax, labels_only=False, subplot_index=None):
    for true_label, probas, label, color in zip(true_labels, probas, labels, colors):
        plot_pr_curve(ax, data, true_label, probas, label, color, labels_only=labels_only, subplot_index=subplot_index)

    no_skill = sum(data[true_labels[0]]) / len(data[true_labels[0]])
    ax.plot([0, 1], [no_skill, no_skill], linestyle='--', color='gray', label=f'{no_skill:.2f}')
    # ax.set_xlabel('Recall', fontsize=16)
    # ax.set_ylabel('Precision', fontsize=16)
    # ax.set_title(f'{title}', fontsize=14)
    # ax.tick_params(axis='x', labelsize=8)
    # ax.tick_params(axis='y', labelsize=8)
    # ax.spines.top.set(visible=False)
    # ax.spines.right.set(visible=False)
    # ax.set_ylim([-0.025, 1.025])
    # ax.set_xlim([-0.025, 1.025])
    # ax.legend(loc='upper right', fontsize=16)

    ax.set_xlabel('Recall', fontsize=12)
    ax.set_ylabel('Precision', fontsize=12)
    ax.set_title(f'{title}', fontsize=12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.spines.top.set(visible=False)
    ax.spines.right.set(visible=False)
    ax.set_ylim([-0.025, 1.025])
    ax.set_xlim([-0.025, 1.025])
    ax.legend(loc='upper right', fontsize=10)

def draw_pr_validation_curves(validation_data_df, title_list, save_path:None):
    sns.set_context('paper')
    fig, ax = plt.subplots(1, 2, figsize=(5.85, 3.75))

    true_labels = ['SL_contextualised', 'SL_full', 'SL_contextualised', 'SL_contextualised']
    predictions = ['ypred_contextualised', 'ypred_full', 'prediction_score', 'min_sequence_identity']
    custom_labels = ['Context-Specific Classifier (contextualised)', 'Context-Specific Classifier (full)', 'General Classifier', 'Sequence Identity']
    colors = ['#009E73', '#CC79A7', '#777777', '#E69F00']

    for i, df in enumerate(validation_data_df):
        draw_pr_curves(df, true_labels, predictions, custom_labels, title_list[i], colors, ax[i], subplot_index=i)

    # Extract handles and labels from the first subplot
    handles, labels = ax[0].get_legend_handles_labels()
    
    # Create a shared legend for the figure
    #fig.legend(handles=handles, labels=custom_labels + ['Chance'], ncol=5, loc='upper center', 
    #           bbox_to_anchor=[0.5, 1.2], fontsize=20, fancybox=True, facecolor='#efefef', borderpad=0.5)

    #fig.legend(handles=handles, labels=custom_labels + ['Chance'], ncol=5, loc='upper center', 
    #           bbox_to_anchor=[0.5, 1.1], fontsize=10, fancybox=True, facecolor='#efefef', borderpad=0.5)

    plt.tight_layout(h_pad=0.5)
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=500)
    plt.close()
    #plt.show()

In [ ]:
# Klingbeil only (indices 0 and 1)
draw_pr_validation_curves(validation_data_df[0:2], 
                          ["Klingbeil et al. - Seen", "Klingbeil et al. - Unseen"],
                          figure_path_pr_auc)

In [41]:
# Parrish only (indices 2 and 3)
draw_pr_validation_curves(validation_data_df[2:4], 
                       ["Parrish et al. - Seen", "Parrish et al. - Unseen"],
                       figure_path_parrish_pr_auc)